In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import Tuple

# --- Lommel–Seeliger 几何因子 ---
def lommel_seeliger(i_deg: np.ndarray, e_deg: np.ndarray) -> np.ndarray:
    i = np.deg2rad(i_deg)
    e = np.deg2rad(e_deg)
    return np.cos(i) / (np.cos(i) + np.cos(e) + 1e-12)

@dataclass
class MMPFParams:
    # 采用 Speyerer(2022) 所用的 7 参数形式（在 Boyd 的 6 参数式基础上常见地多一个项）
    # 你也可以通过设置 a2=0 或去掉某一项来回退到 6 参数
    a0: float = 0.0   # g^2
    a1: float = 0.0   # g
    a2: float = 0.0   # sqrt(g)  （若用 6 参数可将其固定为 0）
    a3: float = 0.0   # cos(e)
    a4: float = 0.0   # cos(i)
    a5: float = 0.0   # cos^2(i)
    a6: float = 0.0   # 常见扩展项（可为 0；若严格 6/7 参数按你的版本切换）

def mmpf_if(i_deg, e_deg, g_deg, p: MMPFParams) -> np.ndarray:
    """
    经验光度函数： I/F = LS * exp( a0*g^2 + a1*g + a2*sqrt(g) + a3*cos(e) + a4*cos(i) + a5*cos(i)^2 + a6 )
    该式与 Boyd(2019)/Speyerer(2022) 的“平滑指数多项式 × LS”思想一致；具体项数可按论文或你数据裁剪:contentReference[oaicite:2]{index=2}:contentReference[oaicite:3]{index=3}。
    """
    i = np.deg2rad(i_deg); e = np.deg2rad(e_deg)
    g = np.asarray(g_deg, dtype=float)
    LS = lommel_seeliger(i_deg, e_deg)
    term = (p.a0 * g**2 +
            p.a1 * g +
            p.a2 * np.sqrt(np.clip(g, 0, None)) +
            p.a3 * np.cos(e) +
            p.a4 * np.cos(i) +
            p.a5 * (np.cos(i)**2) +
            p.a6)
    return LS * np.exp(term)


In [ ]:
import numpy as np
from numpy.linalg import lstsq

def design_matrix(i_deg, e_deg, g_deg, use_sqrt_g=True, with_bias=True, use_cos2i=True):
    i = np.deg2rad(i_deg); e = np.deg2rad(e_deg)
    g = np.asarray(g_deg, dtype=float)
    feats = []
    feats.append(g**2)       # ↔ a0
    feats.append(g)          # ↔ a1
    if use_sqrt_g:
        feats.append(np.sqrt(np.clip(g, 0, None)))  # ↔ a2
    feats.append(np.cos(e))  # ↔ a3
    feats.append(np.cos(i))  # ↔ a4
    if use_cos2i:
        feats.append(np.cos(i)**2)  # ↔ a5
    if with_bias:
        feats.append(np.ones_like(g))  # ↔ a6
    X = np.vstack(feats).T
    return X

def fit_mmpf(i_deg, e_deg, g_deg, IF_obs, use_sqrt_g=True, with_bias=True, use_cos2i=True) -> MMPFParams:
    # 将 LS 因子移到等式左边： log(I/F) - log(LS) = 线性组合
    LS = lommel_seeliger(i_deg, e_deg)
    y = np.log(np.clip(IF_obs, 1e-9, None)) - np.log(np.clip(LS, 1e-12, None))
    X = design_matrix(i_deg, e_deg, g_deg, use_sqrt_g, with_bias, use_cos2i)
    coef, *_ = lstsq(X, y, rcond=None)

    # 回填到参数结构体（按 design_matrix 的顺序）
    idx = 0
    a0 = coef[idx]; idx += 1
    a1 = coef[idx]; idx += 1
    a2 = coef[idx] if use_sqrt_g else 0.0; idx += (1 if use_sqrt_g else 0)
    a3 = coef[idx]; idx += 1
    a4 = coef[idx]; idx += 1
    a5 = coef[idx] if use_cos2i else 0.0; idx += (1 if use_cos2i else 0)
    a6 = coef[idx] if with_bias else 0.0
    return MMPFParams(a0,a1,a2,a3,a4,a5,a6)

def phase_ratio(IF_fn, i1,e1,g1, i2,e2,g2):
    """按 Speyerer(2022) 的做法，计算两组标准几何下的 I/F 之比（如 30° vs 60° 相角比）:contentReference[oaicite:6]{index=6}。"""
    num = IF_fn(i1,e1,g1)
    den = IF_fn(i2,e2,g2) + 1e-12
    return num / den


In [ ]:
# 假设你已有观测角度与 I/F
rng = np.random.default_rng(0)
N = 2000
i_deg = rng.uniform(0, 80, N)
e_deg = rng.uniform(0, 80, N)
g_deg = np.clip(i_deg + e_deg, 0, 140)  # 简易近似：非严格，仅演示

# 生成“真”参数并合成 I/F（用于演示拟合的可行性）
true_p = MMPFParams(a0=-5e-5, a1=-0.01, a2=0.02, a3=0.15, a4=0.10, a5=-0.05, a6=-1.5)
IF_syn = mmpf_if(i_deg,e_deg,g_deg,true_p) * np.exp(rng.normal(0, 0.05, N))  # 5% 对数噪声

# 拟合
est_p = fit_mmpf(i_deg,e_deg,g_deg, IF_syn)
print(est_p)

# 计算 Speyerer 风格的相角比图（例如：I/F( i=30,e=0,g=30 ) / I/F( i=60,e=0,g=60 )）
IF_fn = lambda I,E,G: mmpf_if(np.asarray(I), np.asarray(E), np.asarray(G), est_p)
ratio = phase_ratio(IF_fn, 30,0,30, 60,0,60)
print("Phase ratio (30°/60°) =", float(ratio))


In [ ]:
import torch
from torch import nn
from dataclasses import asdict

class MMPFLayer(nn.Module):
    def __init__(self, init_params: MMPFParams=None, trainable=True, use_sqrt_g=True, with_bias=True, use_cos2i=True):
        super().__init__()
        self.use_sqrt_g = use_sqrt_g
        self.with_bias = with_bias
        self.use_cos2i = use_cos2i

        if init_params is None:
            init_params = MMPFParams()
        vec = [
            init_params.a0, init_params.a1,
            init_params.a2 if use_sqrt_g else 0.0,
            init_params.a3, init_params.a4,
            init_params.a5 if use_cos2i else 0.0,
            init_params.a6 if with_bias else 0.0
        ]
        self.theta = nn.Parameter(torch.tensor(vec, dtype=torch.float32), requires_grad=trainable)

    def forward(self, i_deg: torch.Tensor, e_deg: torch.Tensor, g_deg: torch.Tensor) -> torch.Tensor:
        # LS
        i = torch.deg2rad(i_deg); e = torch.deg2rad(e_deg)
        LS = torch.cos(i) / (torch.cos(i) + torch.cos(e) + 1e-12)

        # 组装设计矩阵（和上面 numpy 版本对应的项序）
        feats = [
            g_deg**2,           # a0
            g_deg,              # a1
        ]
        idx = 2
        if self.use_sqrt_g:
            feats.append(torch.sqrt(torch.clamp(g_deg, min=0.0)))  # a2
            idx += 1
        feats += [
            torch.cos(e),       # a3
            torch.cos(i),       # a4
        ]
        if self.use_cos2i:
            feats.append(torch.cos(i)**2)  # a5
            idx += 1
        if self.with_bias:
            feats.append(torch.ones_like(g_deg))  # a6
            idx += 1

        X = torch.stack(feats, dim=-1)   # (..., K)
        term = torch.sum(X * self.theta, dim=-1)
        IF = LS * torch.exp(term)
        return IF

# —— 演示：用 MSE 拟合参数（固定角度与 I/F 观测）——
def fit_params_torch(i_deg, e_deg, g_deg, IF_obs, steps=2000, lr=5e-2):
    device = "cpu"
    i_t = torch.tensor(i_deg, dtype=torch.float32, device=device)
    e_t = torch.tensor(e_deg, dtype=torch.float32, device=device)
    g_t = torch.tensor(g_deg, dtype=torch.float32, device=device)
    y_t = torch.tensor(IF_obs, dtype=torch.float32, device=device)

    layer = MMPFLayer(trainable=True)
    opt = torch.optim.Adam(layer.parameters(), lr=lr)
    for t in range(steps):
        opt.zero_grad()
        y_hat = layer(i_t, e_t, g_t)
        # 根据 Boyd 做法，也可在 log 域拟合：F = log(I/F) - log(LS)，此处演示直接 MSE
        loss = torch.mean((torch.log(torch.clamp(y_hat, min=1e-9)) - torch.log(torch.clamp(y_t, min=1e-9)))**2)
        loss.backward()
        opt.step()
        if (t+1) % 500 == 0:
            print(f"[{t+1}] loss(log)={loss.item():.5f}")

    # 导出参数
    theta = layer.theta.detach().cpu().numpy().tolist()
    # 还原到 MMPFParams（按开关决定项数）
    # 顺序与上面一致
    idx = 0
    a0 = theta[idx]; idx+=1
    a1 = theta[idx]; idx+=1
    a2 = theta[idx] if layer.use_sqrt_g else 0.0; idx += (1 if layer.use_sqrt_g else 0)
    a3 = theta[idx]; idx+=1
    a4 = theta[idx]; idx+=1
    a5 = theta[idx] if layer.use_cos2i else 0.0; idx += (1 if layer.use_cos2i else 0)
    a6 = theta[idx] if layer.with_bias else 0.0
    return MMPFParams(a0,a1,a2,a3,a4,a5,a6), layer


In [ ]:
est_p_torch, layer = fit_params_torch(i_deg, e_deg, g_deg, IF_syn, steps=1500, lr=0.05)
print(est_p_torch)

# 计算相角比（30/60）
def IF_fn_torch(I,E,G, p: MMPFParams):
    I = np.asarray(I); E = np.asarray(E); G = np.asarray(G)
    return mmpf_if(I,E,G,p)

print("Phase ratio (30°/60°) =", float(phase_ratio(
    lambda I,E,G: IF_fn_torch(I,E,G, est_p_torch),
    30,0,30, 60,0,60)))
